In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import pydicom
import datetime
from pydicom.dataset import Dataset, FileDataset, FileMetaDataset
from pydicom.filewriter import correct_ambiguous_vr

pi = np.pi

In [ ]:
def getSource(alfa, r):
    Xe = r * np.cos(alfa)
    Ye = r * np.sin(alfa)
    
    return (Xe, Ye)

#n - ilosc detektorow, alfa - przesuniecie wszystkeigo, phi - rozstawienie detektorow, r - promien kola
def getDetectors(n, alfa, phi, r):
    detectors = []
    
    # D0
    Xd = r * np.cos(alfa + pi - (phi/2))
    Yd = r * np.sin(alfa + pi - (phi/2))

    detectors.append( (Xd, Yd) )
    
    # D1 ... Dn-1
    for i in range(1,n):
        Xd = r * np.cos(alfa + pi - (phi/2) + (i * (phi/(n-1))))
        Yd = r * np.sin(alfa + pi - (phi/2) + (i * (phi/(n-1))))
        
        detectors.append( (Xd, Yd) )
    
    return detectors

def transformToPictureCoordinates(point, picHeight, picWidth):
    X = point[0] + (0.5*picWidth)
    Y = point[1] + (0.5*picHeight)
    
    X = int(np.floor(X))
    Y = int(np.floor(Y))
    return ( X, Y )

In [ ]:
def bresenhamL(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dy < 0:
        m = -1
        dy = -dy
    j = y1
    e = (2 * dy) - dx
    
    for i in range(x1, x2):
        points.append((i, j))
        if e > 0:
            j += m
            e += 2 * (dy - dx)
        else:
            e += 2 * dy
    return points

def bresenhamH(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dx < 0:
        m = -1
        dx = -dx
    i = x1
    e = (2 * dx) - dy
    
    for j in range(y1, y2):
        points.append((i, j))
        if e > 0:
            i += m
            e += 2 * (dx - dy)
        else:
            e += 2 * dx
    return points

def bresenham(x1, y1, x2, y2):
    if abs(y2 - y1) < abs(x2 - x1):
        if x1 > x2:
            return bresenhamL(x2, y2, x1, y1)
        else:
            return bresenhamL(x1, y1, x2, y2)
    else:
        if y1 > y2:
            return bresenhamH(x2, y2, x1, y1)
        else:
            return bresenhamH(x1, y1, x2, y2)

In [ ]:
def absorption(img, x1, y1, x2, y2):
    height = img.shape[0]
    width = img.shape[1]
    
    s = 0
    
    for (x, y) in bresenham(x1, y1, x2, y2):
        if x >= 0 and x < width and y >= 0 and y < height:
            s += img[x][y]
    return s

In [ ]:
def img2sinogram(img, alfaDelta, n, phi):
    alfaDelta = np.radians(alfaDelta)
    phi = np.radians(phi)
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    d = np.sqrt( picWidth**2 + picHeight**2 )
    r =  d / 2
    
    lines = []
    alfa = 0
    while alfa < pi:
        
        source = getSource(alfa, r)
        pSource = transformToPictureCoordinates(source, picWidth, picHeight)
        
        detectors = getDetectors(n, alfa, phi, r)
        pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
        
        line = []
        
        for detector in pDetectors:
            a = absorption(img, pSource[0], pSource[1], detector[0], detector[1])
            line.append(a)
            
        
        lines.append(line)
        alfa += alfaDelta
    
    maxValue = max(map(max, lines))
    normalized = [[v / maxValue for v in line] for line in lines]
    
    #plt.imshow(normalized, cmap='binary_r')
    return np.stack(normalized, axis=0)

def sinogram2img(sin, picHeight, picWidth, phi, iteration, filtered):
    sinHeight = sin.shape[0]
    sinWidth = sin.shape[1]
    phi = np.radians(phi)
    d = np.sqrt( picWidth**2 + picHeight**2 )
    r =  d / 2
    
    alfaDelta = np.radians(180 / sinHeight)
    n = sinWidth
    
    img = np.zeros((picHeight, picWidth))
    
    alfa = 0
    line = 0
    while alfa < pi and line < iteration:
        
        source = getSource(alfa, r)
        pSource = transformToPictureCoordinates(source, picWidth, picHeight)
        
        detectors = getDetectors(n, alfa, phi, r)
        pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
        #print("D", len(detectors))
        
        for row, detector in enumerate(pDetectors):
            for (x, y) in bresenham(pSource[0], pSource[1], detector[0], detector[1]):
                if x >= 0 and x < picWidth and y >= 0 and y < picHeight:
                    img[y][x] += sin[line][row]
            
        
        alfa += alfaDelta
        line += 1
        
    maxValue = img.max()
    normalized = [[v / maxValue for v in line] for line in img]
    
    plt.imshow(normalized)

In [ ]:
alfaDelta_min = 1
alfaDelta_max = 90
alfaDelta_value = 2

detectors_min = 3
detectors_max = 180
detectors_value = 180

phi_min = 3
phi_max = 180
phi_value = 180

def interactive_getAttributes():
    def getAttributes(alfaDelta, detectors, phi):
        return (alfaDelta, detectors, phi)
    return interactive(getAttributes, alfaDelta  = widgets.IntSlider(min=alfaDelta_min, max=alfaDelta_max, step=1, value=alfaDelta_value), detectors = widgets.IntSlider(min=detectors_min, max=detectors_max, step=1, value=detectors_value), phi = widgets.IntSlider(min=phi_min, max=phi_max, step=1, value=phi_value))
    

def interactive_sinogram(sinogram):
    max_iterations = len(sinogram)
    
    def draw_part_of_sinogram(sinogram, iteration):
        l  = [(line if i < iteration else [0] * len(line)) for i, line in enumerate(sinogram)]
        plt.imshow(l, cmap='binary_r')
        #return iteration

    interact(draw_part_of_sinogram, sinogram = fixed(sinogram), iteration = widgets.IntSlider(min=1, max=max_iterations, step=1, value=max_iterations))


def interactive_output(sinogram, out_size, phi):
    max_iterations = sinogram.shape[0]
    
    interact_manual(sinogram2img, sin = fixed(sinogram), picHeight = fixed(out_size[0]), picWidth = fixed(out_size[1]), phi = fixed(phi), iteration=widgets.IntSlider(min=1, max=max_iterations, step=1, value=max_iterations), filtered=False)

In [ ]:
# n = 4
# phi = np.radians(180)
# alfa = np.radians(90)

# picHeight = img.shape[0]
# picWidth = img.shape[1]
# print("Wysokość:", picHeight, "Szerokość:", picWidth)

# d = np.sqrt( picWidth**2 + picHeight**2 )
# r =  d / 2
# print("Promień: ", r)

# source = getSource(alfa, r)
# pSource = transformToPictureCoordinates(source, picWidth, picHeight)
# print("Source:", pSource)

# detectors = getDetectors(n, alfa, phi, r)
# pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
# print(pDetectors)

In [ ]:
# TESTY

# wspolrzedne abstrakcyjne
source = getSource(pi+pi/4, r)
detectors = getDetectors(3, pi+pi/4, pi/2, r)

# wspolrzedne na obrazku
pSource = transformToPictureCoordinates(source, picWidth, picHeight)
pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]

print(pSource)
print(pDetectors)
# ------------------

In [ ]:
img_name = input('Nazwa pliku: ')

img = mpimg.imread('tomograf-zdjecia/'+img_name)[:,:,2] / 255

# obrazek powinien byc tak
# /\
# |
# |
# |
# (0,0)------->

picHeight = img.shape[0]
picWidth = img.shape[1]
r =  np.sqrt( picWidth**2 + picHeight**2 ) /2


alfaDelta = int(input("Krok układu (stopnie): "))
detectors = int(input("Liczba detektorów: "))
phi = int(input("Rozpiętość detektorów (stopnie): "))

sinogram = img2sinogram(img, alfaDelta, detectors, phi)
interactive_sinogram(sinogram)
interactive_output(sinogram, (picHeight, picWidth), phi)

# Wczytywanie obrazu .jpg

In [ ]:
# Zmienne globalne
img = None
sinogram = None

In [ ]:
wName = widgets.Text(value='Kwadraty2.jpg', description='Nazwa pliku:', disabled=False)

wAlfaDelta = widgets.FloatSlider(value=2, min=0.25, max=2, step=0.25, description="Δα", continuous_update=False)

wN = widgets.IntSlider(value=180, min=90, max=720, step=90, description="L.detektorów", continuous_update=False)

wPhi = widgets.IntSlider(value=180, min=45, max=270, step=45, description="Rozpiętość", continuous_update=False)

wButtonSinogram = widgets.Button(description='Wygeneruj sinogram')

def gen_sinogram(_):
    global wName, wAlfaDelta, wN, wPhi
    global img, sinogram
    img = mpimg.imread('tomograf-zdjecia/'+wName.value)[:,:,2] / 255
    sinogram = img2sinogram(img, wAlfaDelta.value, wN.value, wPhi.value)
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    interactive_sinogram(sinogram)
    interactive_output(sinogram, (picHeight, picWidth), wPhi.value)
    
wButtonSinogram.on_click(gen_sinogram)

display(wName, wAlfaDelta, wN, wPhi, wButtonSinogram)

In [ ]:
sinogram.shape

# Zapis jako DICOM

In [ ]:
def save_dcm(sinogram, filename, date, name, birth, sex, comment):
    file_meta = FileMetaDataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
    file_meta.MediaStorageSOPInstanceUID = "1.2.3"
    file_meta.ImplementationClassUID = "1.2.3.4"
    file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    ds = FileDataset(filename, {}, file_meta=file_meta, preamble=b"\0" * 128)
    ds.PatientName = name
    ds.PatientSex = sex
    ds.PatientBirthDate = birth.strftime('%Y%m%d')
    ds.PatientComments = comment

    ds.is_little_endian = True
    ds.is_implicit_VR = True
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.StudyInstanceUID = pydicom.uid.generate_uid()

    ds.ContentDate = date.strftime('%Y%m%d')
    ds.ContentTime = date.strftime('%H%M%S.%f')

    # 1 lub 3 - ile kolorów na obrazie
    ds.SamplesPerPixel = 1
    # interpretacja (MONOCHROME, RGB, HSV, ...)
    ds.PhotometricInterpretation = "MONOCHROME2"
    # 0 - unsigned int, 1 - U2
    ds.PixelRepresentation = 0
    # najwyższy bit
    ds.HighBit = 15
    # ilość bitów na piksel
    ds.BitsStored = 16
    # ilość bitów zaalokowanych? na piksel
    ds.BitsAllocated = 16
    # minimalna wartość piksela - 0
    ds.SmallestImagePixelValue = str.encode('\x00\x00')
    # maksymalna wartość piksela - ffff?
    ds.LargestImagePixelValue = str.encode('\xff\xff')
    # kolumny, wiersze i dane
    ds.Columns = sinogram.shape[1]
    ds.Rows = sinogram.shape[0]
    ds.PixelData = (sinogram * 255).astype('uint16').tobytes()

    ds = correct_ambiguous_vr(ds, True)
    #print(ds)
    ds.save_as(filename, write_like_original=False)

In [ ]:
wDicomFilename = widgets.Text(value='Kwadraty2.dcm', description='Nazwa pliku')
wDicomPatientName = widgets.Text(value='Jan Kowalski', description='Imię i nazw.')
wDicomPatientBirth = widgets.Text(value='01.01.2000', description='Data ur.')
wDicomPatientBirthLabel = widgets.Label(value='Data powinna być zapisana w formacie DD.MM.YYYY')
wDicomDate = widgets.Text(value=datetime.datetime.now().strftime('%d.%m.%Y %H:%M'), description='Data badania')
wDicomDateLabel = widgets.Label(value='Data powinna być zapisana w formacie DD.MM.YYYY HH:MM')
wDicomPatientSex = widgets.RadioButtons(options=['M', 'F', 'O'], value='O', description='Płeć:')
wDicomComment = widgets.Textarea(placeholder='Wpisz komentarz...', description='Komentarz:')

def wDicomSaveOnClick(_):
    global sinogram
    global wDicomFilename, wDicomDate, wDicomPatientName, wDicomPatientBirth, wDicomPatientSex, wDicomComment
    save_dcm(sinogram,
             wDicomFilename.value,
             datetime.datetime.strptime(wDicomDate.value, '%d.%m.%Y %H:%M'),
             wDicomPatientName.value,
             datetime.datetime.strptime(wDicomPatientBirth.value, '%d.%m.%Y'),
             wDicomPatientSex.value,
             wDicomComment.value)

wDicomSave = widgets.Button(description='Zapisz jako DICOM')
wDicomSave.on_click(wDicomSaveOnClick)

display(wDicomFilename, wDicomPatientName, wDicomPatientBirth, wDicomPatientBirthLabel, wDicomPatientSex)
display(wDicomDate, wDicomDateLabel, wDicomComment, wDicomSave)

# Odczyt DICOM

In [ ]:
def open_dcm(filename):
    global sinogram, ds
    ds = pydicom.dcmread(filename)
    
    print("Imię i nazwisko:", ds.PatientName)
    print("Data urodzenia:", datetime.datetime.strptime(ds.PatientBirthDate, '%Y%m%d').strftime('%d.%m.%Y') )
    print("Płeć:", ds.PatientSex)
    contentDate = datetime.datetime.strptime(ds.ContentDate, '%Y%m%d').strftime('%d.%m.%Y')
    contentTime = datetime.datetime.strptime(ds.ContentTime, '%H%M%S.%f').strftime('%H:%M')
    print("Data badania:", contentDate, contentTime)
    print("Komentarze:", ds.PatientComments)
    sinogram = np.frombuffer(ds.PixelData, dtype='uint16').reshape(ds.Rows, ds.Columns)
    interactive_sinogram(sinogram)

In [ ]:
wDicomFilename2 = widgets.Text(value='Kwadraty2.dcm', description='Nazwa pliku')
wDicomOpen = widgets.Button(description='Otwórz DICOM')

def wDicomOpenOnClick(_):
    global wDicomFilename2
    open_dcm(wDicomFilename2.value)
wDicomOpen.on_click(wDicomOpenOnClick)

display(wDicomFilename2, wDicomOpen)